# Recording a Gesture for Shimi

First, run the cell below to instatiate the shimi object. It should move to its initial positions and then relax.

In [1]:
# Import some matplolib shortcuts for Jupyter notebook
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from shimi import *
from motion.move import *

from motion.recorder import *
from audio.audio import *
from posenet.posenet import *
import time
import datetime

from copy import deepcopy

import os

import threading

# Load Shimi
shimi = Shimi()

local_gestures = {}

Connecting on /dev/tty.usbmodem1431
Found motors with the following IDs: [1, 2, 3, 4, 5]
(OrderedDict([('model', 'MX-28'),
              ('firmware', 30),
              ('id', 1),
              ('baudrate', 1),
              ('return delay time', 0),
              ('angle limit', (-99.03, 14.64)),
              ('highest temperature limit', 80.0),
              ('voltage limit', (6.0, 16.0)),
              ('max torque', 100.0),
              ('status return level', 'always'),
              ('alarm LED', ('Overload Error', 'Overheating Error')),
              ('alarm shutdown', ('Overload Error', 'Overheating Error')),
              ('torque_enable', True),
              ('LED', False),
              ('pid gain', (0.0, 0.0, 4.0)),
              ('goal position', 9.63),
              ('moving speed', 4.788),
              ('torque limit', 100.0),
              ('present position', 9.89),
              ('present speed', 0.0),
              ('present load', 3.1),
              ('present v

The cell below defines the recorder object. The first parameter is a reference to the motor controller, the second is a **list** of motor ids, referenced by a convenience property on `shimi`. The options are as follows:
* `shimi.torso`
* `shimi.neck_lr`
* `shimi.neck_ud`
* `shimi.phone`
* `shimi.foot`
* `shimi.all_motors`: this **returns a list**, so you don't need to wrap it in one in this call


The final parameter is the duration for which you want to record.

In [ ]:
r = Recorder(shimi, shimi.all_motors, 5.0)

This starts a recording. It will count down from 3 to let you know when the recording starts, then move Shimi as you want it to move. **The first thing this call does is relax the motors, so be sure it doesn't collapse on itself when you call this.**

In [ ]:
r.record()
r.plot(plt.axes())

Then, run this to play back the recorded gesture.

In [ ]:
r.play()

### Here are some helper functions.

In [ ]:
# Puts shimi in a neutral position with motors non-compliant
shimi.initial_position()

## **Make sure Shimi will not collapse before you run this!!!**

In [ ]:
# Makes the motors compliant
shimi.disable_torque()

### Here's a hard-coded demo of Shimi dancing to Hey Ya

In [ ]:
play_outkast(shimi)

In [2]:
p = PoseNet(shimi)

Listening for OSC at 127.0.0.1 on port 8000
Starting PoseNet...
Receiving from PoseNet...


In [3]:
p.record()

3.0...
2.0...
1.0...
Recording...
Done. Recorded 20 positions.


In [4]:
p.play()

pos1: 10.627692266158647 pos0: 10.016122421362738 time1: 0.36266231536865234 time0: 0.11950421333312988
pos1: -25.76789738951179 pos0: -24.103275378071316 time1: 0.36266231536865234 time0: 0.11950421333312988
pos1: 10.350573896700254 pos0: 10.627692266158647 time1: 0.6303849220275879 time0: 0.36266231536865234
pos1: -25.754447096813205 pos0: -25.76789738951179 time1: 0.6303849220275879 time0: 0.36266231536865234
pos1: 10.327661175718305 pos0: 10.350573896700254 time1: 0.8694930076599121 time0: 0.6303849220275879
pos1: -32.97450414581529 pos0: -25.754447096813205 time1: 0.8694930076599121 time0: 0.6303849220275879
pos1: 5.893190681136261 pos0: 10.327661175718305 time1: 1.1156182289123535 time0: 0.8694930076599121
pos1: -32.4732519040205 pos0: -32.97450414581529 time1: 1.1156182289123535 time0: 0.8694930076599121
pos1: 2.768541827768189 pos0: 5.893190681136261 time1: 1.3613011837005615 time0: 1.1156182289123535
pos1: -22.71047168020198 pos0: -32.4732519040205 time1: 1.3613011837005615 ti

In [5]:
p.stop_posenet()

Stopping PoseNet...


In [ ]:
m = LinearMove(shimi, shimi.torso, 1.0, 1.0, normalized_positions=True)
m.start()